# Docker "Swarm Mode" Lab

Based on Mario's gist here: https://gist.github.com/l0rd/5186cc80f8f26dc7e9490abca4405830

# Requirements
- Docker 1.12
- Docker machine
- Virtualbox

# Create 3 nodes for your swarm cluster (one master and 2 slaves)

We will create 3 nodes using docker-machine/virtualbox.

![](images/SwarmNodes_3nodes.png)

We will then join them to form a Swarm cluster:
![](images/SwarmNodes_3nodes_1m_2w.png)




#### NOTE:
You may see errors as below, and an error reported by "docker-machine ls", leave some time for the swmaster1 to settle.

In the meantime you can go ahead and create the 2 swnode's below.

In [2]:
MACHINE_DRIVER=virtualbox

CLEANUP_VMS=1
CLEANUP_VMS=0
CLEANUP_VMS=1

. ../NB_bash_functions.rc

Notebook started at Wed Oct 25 07:26:59 CEST 2017   [1508909219]

In [3]:
# Specific to my environment

[ -f ~/.docker.rc ] && . ~/.docker.rc

NB_continue

Setting up alias/DOCKER_HOST for 17.09.0-ce [/home/mjb/DOCKER/docker-17.09.0-ce]

In [4]:
MACHINE_DRIVER=virtualbox

#MACHINE_DRIVER=digitalocean
#MACHINE_DRIVER=azure
#MACHINE_DRIVER=google

Let's cleanup any remaining machines, if they exist already ..

In [5]:
VBoxManage list runningvms | grep sw

echo
docker-machine ls

NAME   ACTIVE   DRIVER   STATE   URL   SWARM   DOCKER   ERRORS

In [6]:
docker-machine ls --filter name=sw --format "{{.Name}}: {{.DriverName}}"

In [7]:
[ $CLEANUP_VMS -ne 0 ] && NB_cleanup_swarm_machines

NB_continue

In [8]:
[ $CLEANUP_VMS -ne 0 ] &&  {
    VBoxManage list runningvms | grep sw

    echo
    docker-machine ls
}

NB_continue

NAME   ACTIVE   DRIVER   STATE   URL   SWARM   DOCKER   ERRORS

Now let's create our new nodes

**NOTE:** You may see some worrying messages from "*docker-machine ls*" but wait a minute or two until the node creation is complete

In [9]:
NB_create_swarm_machine_m1

Running command <docker-machine create -d virtualbox swmaster1> ...
Running pre-create checks...
Creating machine...
(swmaster1) Copying /home/mjb/.docker/machine/cache/boot2docker.iso to /home/mjb/.docker/machine/machines/swmaster1/boot2docker.iso...
(swmaster1) Creating VirtualBox VM...
(swmaster1) Creating SSH key...
(swmaster1) Starting the VM...
(swmaster1) Check network to re-create if needed...
(swmaster1) Waiting for an IP...
Waiting for machine to be running, this may take a few minutes...
Detecting operating system of created instance...
Waiting for SSH to be available...
Detecting the provisioner...
Provisioning with boot2docker...
Copying certs to the local machine directory...
Copying certs to the remote machine...
Setting Docker configuration on the remote daemon...

This machine has been allocated an IP address, but Docker Machine could not
reach it successfully.

SSH for the machine should still work, but connecting to exposed ports, such as
the Docker daemon port (usua

#### NOTE: Temporary errors from "*docker-machine ls*"

From the command-line you can see the progress of machine creation, using
    ```$ docker-machine ls```
    
You may see errors during creation of machines before ssh connectivity is established and before docker host is started.

![](images/docker-machine-errors.png)

After creation of a node you should see something like the following from "*docker-machine ls*"

In [10]:
docker-machine ls

NAME        ACTIVE   DRIVER       STATE     URL                         SWARM   DOCKER        ERRORS
swmaster1   -        virtualbox   Running   tcp://192.168.99.100:2376           v17.10.0-ce

We can also see the VM from virtualbox point of view (if this is the driver we are using):

In [11]:
VBoxManage list runningvms | grep sw

"swmaster1" {881c5a3b-2fd8-4a2d-9a17-92b5746a12a7}

In [12]:
NB_create_swarm_machine_1

Running command <docker-machine create -d virtualbox swnode1> ...
Running pre-create checks...
Creating machine...
(swnode1) Copying /home/mjb/.docker/machine/cache/boot2docker.iso to /home/mjb/.docker/machine/machines/swnode1/boot2docker.iso...
(swnode1) Creating VirtualBox VM...
(swnode1) Creating SSH key...
(swnode1) Starting the VM...
(swnode1) Check network to re-create if needed...
(swnode1) Waiting for an IP...
Waiting for machine to be running, this may take a few minutes...
Detecting operating system of created instance...
Waiting for SSH to be available...
Detecting the provisioner...
Provisioning with boot2docker...
Copying certs to the local machine directory...
Copying certs to the remote machine...
Setting Docker configuration on the remote daemon...
Checking connection to Docker...
Docker is up and running!
To see how to connect your Docker Client to the Docker Engine running on this virtual machine, run: docker-machine env swnode1
Took 62 secs [0h01m02]

In [13]:
VBoxManage list runningvms | grep sw

echo
docker-machine ls

"swmaster1" {881c5a3b-2fd8-4a2d-9a17-92b5746a12a7}
"swnode1" {e7f89252-a01d-4388-96a2-f058d3f53b73}

NAME        ACTIVE   DRIVER       STATE     URL                         SWARM   DOCKER        ERRORS
swmaster1   -        virtualbox   Running   tcp://192.168.99.100:2376           v17.10.0-ce   
swnode1     -        virtualbox   Running   tcp://192.168.99.101:2376           v17.10.0-ce

In [14]:
NB_create_swarm_machine_2

Running command <docker-machine create -d virtualbox swnode2> ...
Running pre-create checks...
Creating machine...
(swnode2) Copying /home/mjb/.docker/machine/cache/boot2docker.iso to /home/mjb/.docker/machine/machines/swnode2/boot2docker.iso...
(swnode2) Creating VirtualBox VM...
(swnode2) Creating SSH key...
(swnode2) Starting the VM...
(swnode2) Check network to re-create if needed...
(swnode2) Waiting for an IP...
Waiting for machine to be running, this may take a few minutes...
Detecting operating system of created instance...
Waiting for SSH to be available...
Detecting the provisioner...
Provisioning with boot2docker...
Copying certs to the local machine directory...
Copying certs to the remote machine...
Setting Docker configuration on the remote daemon...
Checking connection to Docker...
Docker is up and running!
To see how to connect your Docker Client to the Docker Engine running on this virtual machine, run: docker-machine env swnode2
Took 62 secs [0h01m02]

In [15]:
VBoxManage list runningvms | grep sw

echo
docker-machine ls

"swmaster1" {881c5a3b-2fd8-4a2d-9a17-92b5746a12a7}
"swnode1" {e7f89252-a01d-4388-96a2-f058d3f53b73}
"swnode2" {78c1be50-ec93-4c76-b662-fded9d9efb00}

NAME        ACTIVE   DRIVER       STATE     URL                         SWARM   DOCKER        ERRORS
swmaster1   -        virtualbox   Running   tcp://192.168.99.100:2376           v17.10.0-ce   
swnode1     -        virtualbox   Running   tcp://192.168.99.101:2376           v17.10.0-ce   
swnode2     -        virtualbox   Running   tcp://192.168.99.102:2376           v17.10.0-ce

In [16]:
NB_check_3_machines_running

### We have now created 3 docker hosts, which are operating independently for the moment:


![](images/SwarmNodes_3nodes.png)



### Directing the docker client to a particular nodes' docker daemon

**NOTE**: See that we precede all docker commands with $(docker-machine config NODE) where node is the name of the node to which we want our docker client to connect to.  This command returns the parameters to direct our client to the appropriate node.  Run alone this produces:

In [17]:
docker-machine config swmaster1

--tlsverify
--tlscacert="/home/mjb/.docker/machine/machines/swmaster1/ca.pem"
--tlscert="/home/mjb/.docker/machine/machines/swmaster1/cert.pem"
--tlskey="/home/mjb/.docker/machine/machines/swmaster1/key.pem"
-H=tcp://192.168.99.100:2376

We can also obtain these values as environment variables

In [18]:
docker-machine env swmaster1

export DOCKER_TLS_VERIFY="1"
export DOCKER_HOST="tcp://192.168.99.100:2376"
export DOCKER_CERT_PATH="/home/mjb/.docker/machine/machines/swmaster1"
export DOCKER_MACHINE_NAME="swmaster1"
# Run this command to configure your shell: 
# eval $(docker-machine env swmaster1)

Including these parameters on the docker command line will connect the client to the docker daemon running on node '*swmaster1*'.

# Creating the swarm cluster

We now want to move our 3 nodes from being independent:
![](images/SwarmNodes_3nodes.png)

to being 1 Master and 2 Worker Swarm Nodes:
![](images/SwarmNodes_3nodes_1m_2w.png)



### Networks before creation of swarm cluster
Before going further let's look at the networks on your machine.

Later, we'll see how a new network is created once the swarm cluster has been created.

In [19]:
docker $(docker-machine config swmaster1) network ls

NETWORK ID          NAME                DRIVER              SCOPE
ef5c4f493b33        bridge              bridge              local
83c8e3bcc515        host                host                local
726ad7e23324        none                null                local

Note that we already see **3 networks** each of a different **type**:
- **host**: this is the network of your host machine (the network on your host swmaster1)
- **bridge**: this is a separate network on which containers will have their own ip, separated from the host network
- **none/null**:

Now let's identify the ip address of our master node.

We can see this through config or ip commands of docker-machine as shown below.

In [20]:
docker-machine ip swmaster1

192.168.99.100

We could then provide the above ip address as parameter to --advertise-addr when initializing the swarm.

However, it is quite convenient to run the above commands embedded, as below, as arguments to the swarm init command.

docker-machine config swmaster1 provides the parameters to use when connecting to the appropriate docker engine for our machine "swmaster1".

The following command will run swarm init to generate the cluster with 'swmaster1' as the Master node.
You should see output similar to the below:

In [21]:
docker $(docker-machine config swmaster1) info | grep Swarm

Swarm: inactive

# swarm init

Now that we have 3 nodes available, we will initialize our Swarm Cluster with 1 master node.

In [22]:
which docker

alias docker='/home/mjb/DOCKER/docker-17.09.0-ce/docker'
	~/DOCKER/docker-17.09.0-ce/docker

In [23]:
docker $(docker-machine config swmaster1) swarm init --advertise-addr $(docker-machine ip swmaster1)

Swarm initialized: current node (2gfwhg91762xhx532ikjyfobi) is now a manager.

To add a worker to this swarm, run the following command:

    docker swarm join --token SWMTKN-1-0swaxpr36avccy1k4tvr0mnttl25vv5mh1c4uzrsgvu3l6xsso-f08vs0pxhq199utnef9lwyyws 192.168.99.100:2377

To add a manager to this swarm, run 'docker swarm join-token manager' and follow the instructions.

![](images/SwarmNodes_3nodes_1m.png)

A docker info should now show "Swarm: active" as below:

In [24]:
docker $(docker-machine config swmaster1) info | grep Swarm:

Swarm: active

In [25]:
docker $(docker-machine config swmaster1) info

Containers: 0
 Running: 0
 Paused: 0
 Stopped: 0
Images: 0
Server Version: 17.10.0-ce
Storage Driver: aufs
 Root Dir: /mnt/sda1/var/lib/docker/aufs
 Backing Filesystem: extfs
 Dirs: 0
 Dirperm1 Supported: true
Logging Driver: json-file
Cgroup Driver: cgroupfs
Plugins:
 Volume: local
 Network: bridge host macvlan null overlay
 Log: awslogs fluentd gcplogs gelf journald json-file logentries splunk syslog
Swarm: active
 NodeID: 2gfwhg91762xhx532ikjyfobi
 Is Manager: true
 ClusterID: 54w1v9hafus9z0im67mqph98k
 Managers: 1
 Nodes: 1
 Orchestration:
  Task History Retention Limit: 5
 Raft:
  Snapshot Interval: 10000
  Number of Old Snapshots to Retain: 0
  Heartbeat Tick: 1
  Election Tick: 3
 Dispatcher:
  Heartbeat Period: 5 seconds
 CA Configuration:
  Expiry Duration: 3 months
  Force Rotate: 0
 Autolock Managers: false
 Root Rotation In Progress: false
 Node Address: 192.168.99.100
 Manager Addresses:
  192.168.99.100:2377
Runtimes: runc
Default Runtime: runc
Init Binary: docker-init
co

If we look at the networks we should now see new networks such as '*ingress*' an overlay network and docker_gwbridge for the swarm cluster.

In [26]:
docker $(docker-machine config swmaster1) network ls

NETWORK ID          NAME                DRIVER              SCOPE
ef5c4f493b33        bridge              bridge              local
730be762c09b        docker_gwbridge     bridge              local
83c8e3bcc515        host                host                local
rkp0090958gl        ingress             overlay             swarm
726ad7e23324        none                null                local

We see that 2 new networks have been created of type bridge and overlay
- The **ingress** network is a special overlay network that facilitates **load balancing** among a service’s nodes
- The **docker_gwbridge** is a bridge network that connects the overlay networks (including the ingress network) to an individual Docker daemon’s physical network.

By default, each container a service is running is connected to its local Docker daemon host’s docker_gwbridge network.

# swarm join

Now we wish to join Master and Worker nodes to our swarm cluster, to do this we need to obtain the token generated during the "swarm init".

Let's save that token to an environment variable as follows:

In [27]:
worker_token=$(docker $(docker-machine config swmaster1) swarm join-token worker -q)

In [28]:
echo $worker_token

SWMTKN-1-0swaxpr36avccy1k4tvr0mnttl25vv5mh1c4uzrsgvu3l6xsso-f08vs0pxhq199utnef9lwyyws

Now we can use this token to join nodes as a worker to this cluster

Note: we could also join nodes as Master, but we have only 3 nodes available.

Let's join swnode1 as a worker node

In [29]:
docker $(docker-machine config swnode1) swarm join --token $worker_token $(docker-machine ip swmaster1):2377

This node joined a swarm as a worker.

![](images/SwarmNodes_3nodes_1m_1w.png)

Now we can use the same token to join node swnode2 as a worker node


In [30]:
docker $(docker-machine config swnode2) swarm join --token $worker_token $(docker-machine ip swmaster1):2377

This node joined a swarm as a worker.

![](images/SwarmNodes_3nodes_1m_2w.png)


In [31]:
docker $(docker-machine config swmaster1) node ls

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
2gfwhg91762xhx532ikjyfobi *   swmaster1           Ready               Active              Leader
21m2hzus2bqe7y81hkstiyeo4     swnode1             Ready               Active              
ps52jax88o8wd1d6v9heoswm4     swnode2             Ready               Active

# start service

First we check for any running services - there should be none in our newly initialized cluster:

In [32]:
docker $(docker-machine config swmaster1) service ls

ID                  NAME                MODE                REPLICAS            IMAGE               PORTS

Now we will create a new service based on the docker image mjbright/docker-demo

We will expose this service on port 8080


In [33]:
docker $(docker-machine config swmaster1) service create --detach --replicas 1 --name docker-demo -p 8080:8080 mjbright/docker-demo:20

mjh3fk6bc1ish5cpxa2wmn1nw
Since --detach=false was not specified, tasks will be created in the background.
In a future release, --detach=false will become the default.

Now we list services again and we should see our newly added docker-demo service

In [34]:
docker $(docker-machine config swmaster1) service ls

ID                  NAME                MODE                REPLICAS            IMAGE                     PORTS
mjh3fk6bc1is        docker-demo         replicated          1/1                 mjbright/docker-demo:20   *:8080->8080/tcp

In [35]:
docker $(docker-machine config swmaster1) service ls

ID                  NAME                MODE                REPLICAS            IMAGE                     PORTS
mjh3fk6bc1is        docker-demo         replicated          1/1                 mjbright/docker-demo:20   *:8080->8080/tcp

In [36]:
NB_docker_loop_until_service_started docker-demo

mjh3fk6bc1is        docker-demo         replicated          1/1                 mjbright/docker-demo:20   *:8080->8080/tcp

it may take a few seconds until the service has a running container (see REPLICAS column) especially if the container image has not yet been downloaded

In [37]:
docker $(docker-machine config swmaster1) service ls

ID                  NAME                MODE                REPLICAS            IMAGE                     PORTS
mjh3fk6bc1is        docker-demo         replicated          1/1                 mjbright/docker-demo:20   *:8080->8080/tcp

... and we can look at the service as seen by the cluster:

In [38]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID                  NAME                IMAGE                     NODE                DESIRED STATE       CURRENT STATE            ERROR               PORTS
qzj80y51tinx        docker-demo.1       mjbright/docker-demo:20   swnode1             Running             Running 39 seconds ago

... and we can look at the service on the individual cluster nodes.

Of course as we set replicas to 1 there is only 1 replica of the service for the moment, running on just 1 node of our cluster:

In [39]:
docker $(docker-machine config swmaster1) ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES

In [40]:
docker $(docker-machine config swnode1) ps

CONTAINER ID        IMAGE                     COMMAND                  CREATED              STATUS              PORTS               NAMES
c29fea36fe77        mjbright/docker-demo:20   "/bin/docker-demo ..."   About a minute ago   Up About a minute   8080/tcp            docker-demo.1.qzj80y51tinx8b5glq07rkiuz

In [41]:
docker $(docker-machine config swnode2) ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES

# Accessing the service

As we are working remotely we need to create an ssh tunnel through to the swarm cluster to access our service, exposing the port 8080 on your local machine.

We can obtain the ip address of the swarm master node as follows.

In [48]:
NODE=swnode1

IP=$(docker-machine ip $NODE)

echo $IP

echo "Connect to http://${IP}:8080"

192.168.99.101
Connect to http://192.168.99.101:8080

Let's set an environment variable with that ip address

In [47]:
MASTERIP=$(docker-machine ip swmaster1)

Then open your web browser at the above page [http://IP:8080](http://${IP}:8080) (replace <IP> with the appropriate value) and you should see a lovely blue whale, as below:

![](images/docker.png)


Alternatively we can connect using a command-line client (the web server detects if it's a wget or curl request and returns ASCII text instead of an image).

In [49]:
wget -O - http://$IP:8080

--2017-10-25 09:40:00--  http://192.168.99.101:8080/
Connecting to 192.168.99.101:8080... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/txt]
Saving to: ‘STDOUT’

-                       [<=>                 ]       0  --.-KB/s               


                                                .---------.                                          
                                               .///++++/:.                                          
                                               .///+++//:.                                          
                                               .///+++//:.                                          
                             ``````````````````.:///////:.                       `                  
                             .-///////:://+++//::///////-.                      .--.                
                             .:::///:::///+++///:::///:::.                     .:ss+-`              
          

# scale service

Now we can scale the service to 3 replicas:

In [50]:
docker $(docker-machine config swmaster1) service scale docker-demo=3

docker-demo scaled to 3
Since --detach=false was not specified, tasks will be scaled in the background.
In a future release, --detach=false will become the default.

In [51]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID                  NAME                IMAGE                     NODE                DESIRED STATE       CURRENT STATE             ERROR               PORTS
qzj80y51tinx        docker-demo.1       mjbright/docker-demo:20   swnode1             Running             Running 8 minutes ago                         
xduwdjcfsrvv        docker-demo.2       mjbright/docker-demo:20   swmaster1           Running             Preparing 6 seconds ago                       
wwac7sqcekq3        docker-demo.3       mjbright/docker-demo:20   swnode2             Running             Preparing 6 seconds ago

# rolling-update

Now we will see how we can perform a rolling update.

We initially deployed version 20 of the service, now we will upgrade our whole cluster to version 21


In [52]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID                  NAME                IMAGE                     NODE                DESIRED STATE       CURRENT STATE                    ERROR               PORTS
qzj80y51tinx        docker-demo.1       mjbright/docker-demo:20   swnode1             Running             Running 8 minutes ago                                
xduwdjcfsrvv        docker-demo.2       mjbright/docker-demo:20   swmaster1           Running             Running less than a second ago                       
wwac7sqcekq3        docker-demo.3       mjbright/docker-demo:20   swnode2             Running             Running less than a second ago

In [53]:
docker $(docker-machine config swmaster1) service update --image mjbright/docker-demo:21 docker-demo

docker-demo
Since --detach=false was not specified, tasks will be updated in the background.
In a future release, --detach=false will become the default.

In [ ]:
docker $(docker-machine config swmaster1) pull mjbright/docker-demo:21

In [1]:
docker $(docker-machine config swnode1) pull mjbright/docker-demo:21

21: Pulling from mjbright/docker-demo
Digest: sha256:dd88f9ba2795e3da1597005cdda902fa6d412cd1c435e3897e753c11f4cc902f
Status: Downloaded newer image for mjbright/docker-demo:21

In [2]:
docker $(docker-machine config swnode2) pull mjbright/docker-demo:21

21: Pulling from mjbright/docker-demo
Digest: sha256:dd88f9ba2795e3da1597005cdda902fa6d412cd1c435e3897e753c11f4cc902f
Status: Downloaded newer image for mjbright/docker-demo:21

In [54]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID                  NAME                IMAGE                     NODE                DESIRED STATE       CURRENT STATE                      ERROR               PORTS
i92770901fs7        docker-demo.1       mjbright/docker-demo:21   swnode1             Ready               Preparing less than a second ago                       
qzj80y51tinx         \_ docker-demo.1   mjbright/docker-demo:20   swnode1             Shutdown            Running less than a second ago                         
xduwdjcfsrvv        docker-demo.2       mjbright/docker-demo:20   swmaster1           Running             Running 21 seconds ago                                 
wwac7sqcekq3        docker-demo.3       mjbright/docker-demo:20   swnode2             Running             Running 21 seconds ago

Again it may take a few moments until the new image is downloaded and operational on all nodes.

Once the new image has been downloaded and started on all nodes you should see something like:

In [3]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID            NAME               IMAGE                    NODE       DESIRED STATE  CURRENT STATE                ERROR  PORTS
i92770901fs7  docker-demo.1      mjbright/docker-demo:21  swnode1    Running        Running 3 minutes ago               
qzj80y51tinx   \_ docker-demo.1  mjbright/docker-demo:20  swnode1    Shutdown       Shutdown 4 minutes ago              
ywlc765ichqk  docker-demo.2      mjbright/docker-demo:21  swmaster1  Running        Running 22 seconds ago              
xduwdjcfsrvv   \_ docker-demo.2  mjbright/docker-demo:20  swmaster1  Shutdown       Shutdown about a minute ago         
f4sj0i4i7te2  docker-demo.3      mjbright/docker-demo:21  swnode2    Running        Running about a minute ago          
wwac7sqcekq3   \_ docker-demo.3  mjbright/docker-demo:20  swnode2    Shutdown       Shutdown 3 minutes ago

Now connect again to

In [6]:
MASTERIP=$(docker-machine ip swmaster1)
echo "http://${MASTERIP}:8080"

http://192.168.99.100:8080

### Verifying the service has been updated

Then open your web browser at the page http://MASTERIP:8080 and you should now see a lovely **red** whale.


![](images/docker_red.png)

of if you prefer the command-line ..

In [10]:
#. ../NB_bash_functions.rc

NB_docker_loop_until_N_replicas_started docker-demo 3

#NB_pause

Notebook started at Wed Oct 25 09:47:46 CEST 2017   [1508917666]
mjh3fk6bc1is  docker-demo  replicated  3/3       mjbright/docker-demo:21
Took 1 secs

In [11]:
docker $(docker-machine config swmaster1) service ls

ID            NAME         MODE        REPLICAS  IMAGE
mjh3fk6bc1is  docker-demo  replicated  3/3       mjbright/docker-demo:21

In [12]:
NB_docker_loop_until_service_started docker-demo

mjh3fk6bc1is  docker-demo  replicated  3/3       mjbright/docker-demo:21

In [13]:
wget -O - http://$MASTERIP:8080

--2017-10-25 09:48:11--  http://192.168.99.100:8080/
Connecting to 192.168.99.100:8080... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/txt]
Saving to: ‘STDOUT’

-                       [<=>                 ]       0  --.-KB/s               


                                                .---------.                                          
                                               .///++++/:.                                          
                                               .///+++//:.                                          
                                               .///+++//:.                                          
                             ``````````````````.:///////:.                       `                  
                             .-///////:://+++//::///////-.                      .--.                
                             .:::///:::///+++///:::///:::.                     .:ss+-`              
          

## Service rollback

Revert changes to a service’s configuration

#### API 1.31+
The client and daemon API must both be at least 1.31 to use this command.

Use the docker version command on the client to check your client and daemon API versions.

In [17]:
docker version

Client:
 Version:         1.13.1
 API version:     1.26
 Package version: <unknown>
 Go version:      go1.8.3
 Git commit:      b5e3294/1.13.1
 Built:           Wed Aug 16 13:58:20 2017
 OS/Arch:         linux/amd64

Server:
 Version:         1.13.1
 API version:     1.26 (minimum version 1.12)
 Package version: <unknown>
 Go version:      go1.8.3
 Git commit:      b5e3294/1.13.1
 Built:           Wed Aug 16 13:58:20 2017
 OS/Arch:         linux/amd64
 Experimental:    false

In [16]:
docker service rollback docker-demo

Usage:	docker service COMMAND

Manage services

Options:
      --help   Print usage

Commands:
  create      Create a new service
  inspect     Display detailed information on one or more services
  logs        Fetch the logs of a service
  ls          List services
  ps          List the tasks of a service
  rm          Remove one or more services
  scale       Scale one or multiple replicated services
  update      Update a service

Run 'docker service COMMAND --help' for more information on a command.

# drain a node

We can drain a node effectively placing it in 'maintenance mode'.

Draining a node means that it no longer has running tasks on it.

In [18]:
docker $(docker-machine config swmaster1) node ls

ID                           HOSTNAME   STATUS  AVAILABILITY  MANAGER STATUS
21m2hzus2bqe7y81hkstiyeo4    swnode1    Ready   Active        
2gfwhg91762xhx532ikjyfobi *  swmaster1  Ready   Active        Leader
ps52jax88o8wd1d6v9heoswm4    swnode2    Ready   Active

Let's drain swnode1

In [19]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID            NAME               IMAGE                    NODE       DESIRED STATE  CURRENT STATE            ERROR  PORTS
i92770901fs7  docker-demo.1      mjbright/docker-demo:21  swnode1    Running        Running 10 minutes ago          
qzj80y51tinx   \_ docker-demo.1  mjbright/docker-demo:20  swnode1    Shutdown       Shutdown 11 minutes ago         
ywlc765ichqk  docker-demo.2      mjbright/docker-demo:21  swmaster1  Running        Running 7 minutes ago           
xduwdjcfsrvv   \_ docker-demo.2  mjbright/docker-demo:20  swmaster1  Shutdown       Shutdown 8 minutes ago          
f4sj0i4i7te2  docker-demo.3      mjbright/docker-demo:21  swnode2    Running        Running 8 minutes ago           
wwac7sqcekq3   \_ docker-demo.3  mjbright/docker-demo:20  swnode2    Shutdown       Shutdown 10 minutes ago

In [20]:
docker $(docker-machine config swmaster1) node update --availability drain swnode1

swnode1

and now we see that all services on swnode1 are shutdown

In [21]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID            NAME               IMAGE                    NODE       DESIRED STATE  CURRENT STATE            ERROR  PORTS
p1x5e0s4ppip  docker-demo.1      mjbright/docker-demo:21  swmaster1  Ready          Ready 5 seconds ago             
i92770901fs7   \_ docker-demo.1  mjbright/docker-demo:21  swnode1    Shutdown       Running 5 seconds ago           
qzj80y51tinx   \_ docker-demo.1  mjbright/docker-demo:20  swnode1    Shutdown       Shutdown 11 minutes ago         
ywlc765ichqk  docker-demo.2      mjbright/docker-demo:21  swmaster1  Running        Running 7 minutes ago           
xduwdjcfsrvv   \_ docker-demo.2  mjbright/docker-demo:20  swmaster1  Shutdown       Shutdown 8 minutes ago          
f4sj0i4i7te2  docker-demo.3      mjbright/docker-demo:21  swnode2    Running        Running 9 minutes ago           
wwac7sqcekq3   \_ docker-demo.3  mjbright/docker-demo:20  swnode2    Shutdown       Shutdown 10 minutes ago

But as we had request 3 replicas of our service the Swarm mode will take care to start extra instances on other nodes so that we still have 3 replicas of the service running.

So after a few seconds we see that we once again have 3 replicas in the *Running* state, as shown below:

In [22]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID            NAME               IMAGE                    NODE       DESIRED STATE  CURRENT STATE            ERROR  PORTS
p1x5e0s4ppip  docker-demo.1      mjbright/docker-demo:21  swmaster1  Running        Running 17 seconds ago          
i92770901fs7   \_ docker-demo.1  mjbright/docker-demo:21  swnode1    Shutdown       Shutdown 18 seconds ago         
qzj80y51tinx   \_ docker-demo.1  mjbright/docker-demo:20  swnode1    Shutdown       Shutdown 12 minutes ago         
ywlc765ichqk  docker-demo.2      mjbright/docker-demo:21  swmaster1  Running        Running 8 minutes ago           
xduwdjcfsrvv   \_ docker-demo.2  mjbright/docker-demo:20  swmaster1  Shutdown       Shutdown 9 minutes ago          
f4sj0i4i7te2  docker-demo.3      mjbright/docker-demo:21  swnode2    Running        Running 9 minutes ago           
wwac7sqcekq3   \_ docker-demo.3  mjbright/docker-demo:20  swnode2    Shutdown       Shutdown 11 minutes ago

# remove a service

Now let's cleanup by removing our service

In [23]:
docker $(docker-machine config swmaster1) service rm docker-demo

docker-demo

We can check that the service is no longer running:

In [24]:
docker $(docker-machine config swmaster1) service ps docker-demo

NB_continue

Error: No such service: docker-demo

In [25]:
docker $(docker-machine config swmaster1) ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES

# Deploying a stack

Since Docker 1.12 there is the concept of a Docker Stack.

Docker Stack, similar to Docker-compose allows to deploy a whole application made up of a group of components and especially a group of services.

Stacks extend this concept providing more features such as scaling and node placement.

Let's deploy a stack

First let's clone Alex Ellis' OpenFaaS

(FaaS = Function as a Service - a Docker implementation of "Serverless Computing").

https://github.com/alexellis/faas

There is a quick test drive document here:
https://github.com/alexellis/faas/blob/master/TestDrive.md

from which the following steps were taken:

In [27]:
git clone https://github.com/alexellis/faas alexellis.faas
cd alexellis.faas

Cloning into 'alexellis.faas'...
remote: Counting objects: 11824, done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 11824 (delta 4), reused 11 (delta 3), pack-reused 11808
Receiving objects: 100% (11824/11824), 16.70 MiB | 465.00 KiB/s, done.
Resolving deltas: 100% (3504/3504), done.

Take the time to look at the docker-compose.yml file.

Note that this is the latest version 3 with extra stack capabilities such as deploy/placement tags.

In [28]:
cat docker-compose.yml

version: "3.3"
services:
    gateway:
        volumes:
            - "/var/run/docker.sock:/var/run/docker.sock"
        ports:
            - 8080:8080
        image: functions/gateway:0.6.7
        networks:
            - functions
        environment:
            dnsrr: "true"  # Temporarily use dnsrr in place of VIP while issue persists on PWD
        deploy:
            placement:
                constraints:
                    - 'node.role == manager'
                    - 'node.platform.os == linux'
    prometheus:
        image: functions/prometheus:latest  # autobuild from Dockerfile in repo.
        command: "-config.file=/etc/prometheus/prometheus.yml -storage.local.path=/prometheus -storage.local.memory-chunks=10000 --alertmanager.url=http://alertmanager:9093"
        ports:
            - 9090:9090
        depends_on:
            - gateway
            - alertmanager
        environment:
            no_proxy: "gateway"
        networks:
            - functions
        deploy

In [29]:
cat deploy_stack.sh

#!/bin/sh

if ! [ -x "$(command -v docker)" ]; then
  echo 'Unable to find docker command, please install Docker (https://www.docker.com/) and retry' >&2
  exit 1
fi

echo "Deploying stack"
docker stack deploy func --compose-file docker-compose.yml

**Note: if running on play-with-docker.com**, copy-paste the following directly

```
      docker swarm init --advertise-addr eth0 && \
        git clone https://github.com/alexellis/faas && \
        cd faas && \
        ./deploy_stack.sh && \
        docker service ls
```

**Note**: Running under play-with-docker.com you will have access to Prometheus also

In [33]:
which docker-machine

docker-machine version

/usr/local/bin/docker-machine
docker-machine version 0.13.0, build 9ba6da9

In [32]:
eval $(docker-machine env swmaster1)

time ./deploy_stack.sh

Deploying stack
unsupported Compose file version: 3.3

real	0m0.036s
user	0m0.025s
sys	0m0.012s

In [40]:
git checkout 17cd3d8028f2096fdecc7ea9f5813af17c1bb1cb

Previous HEAD position was 58cbec9... Contribution ideas
HEAD is now at 17cd3d8... Added syntax highlighting

In [41]:
eval $(docker-machine env swmaster1)

time ./deploy_stack.sh

Deploying stack
Creating network func_functions
Creating service func_webhookstash
Creating service func_gateway
Creating service func_nodeinfo
Creating service func_hubstats
Creating service func_markdown
Creating service func_echoit
Creating service func_base64
Creating service func_alertmanager
Creating service func_wordcount
Creating service func_decodebase64
Creating service func_prometheus

real	0m0.090s
user	0m0.040s
sys	0m0.015s

If you are running on PWD, click on the link to port 8080 at the top of the page.

If you are running via docker-machine open an ssh tunnel to forward this port locally
    docker-machine ssh swmaster1 -L 8080:localhost:8080

### Connect to FaaS dashboard
Then connect to http://localhost:8080 to see the FaaS dashboard

In [43]:
echo "http://${MASTERIP}:8080"

http://192.168.99.100:8080

### Inspect your FaaS implementation

From the command-line inspect the stack you've just deployed using
- docker service ls
- docker service ps &lt;service-name&gt;

### Play with FaaS
The point of this exercise was to demonstrate the deploying of the stack, but why not play with FaaS a bit now, you deserve it !

But don't spend too long, you've a Docker-Python class to do next !!

## Visualizing the stack

We can use Mano Marks' visualizer application to view our deployed stack.

Let's launch the visualizer on swmaster1, exposed on port 7070

In [44]:
docker $(docker-machine config swmaster1) run -it -d -p 7070:8080 -v /var/run/docker.sock:/var/run/docker.sock dockersamples/visualizer

Unable to find image 'dockersamples/visualizer:latest' locally
latest: Pulling from dockersamples/visualizer













Digest: sha256:3136807ad7b92fddc5eb45689df93c45907bb9047b52f5f5960a9605ed7536d1
Status: Downloaded newer image for dockersamples/visualizer:latest
cae2e21a6ff9489ff507a45db82d236324b899c54149799234def72e61f3d0dd

we can now connect to the stack at swmaster1:7070

In [45]:
MASTERIP=$(docker-machine ip swmaster1)

echo "Connect to http://$MASTERIP:7070"

Connect to http://192.168.99.100:7070

You should see something like this

![](images/visualizer.png)

In [51]:
docker $(docker-machine config swmaster1) ps

CONTAINER ID        IMAGE                           COMMAND                  CREATED             STATUS                   PORTS                    NAMES
cae2e21a6ff9        dockersamples/visualizer        "npm start"              5 minutes ago       Up 5 minutes (healthy)   0.0.0.0:7070->8080/tcp   goofy_fermi
12c3253112b9        functions/nodeinfo:latest       "fwatchdog"              8 minutes ago       Up 8 minutes (healthy)                            func_nodeinfo.1.k427lc7b38vgx9nqy7mnyafxv
3b980dabbabb        functions/alpine:health         "fwatchdog"              8 minutes ago       Up 8 minutes (healthy)                            func_base64.1.utuqkr6vm69jmvqhhz046fp3r
3347b5eaa594        functions/prometheus:latest     "/bin/prometheus -..."   9 minutes ago       Up 9 minutes             9090/tcp                 func_prometheus.1.r1xft54pyq3j5cdg1qy3vuq7c
8cfc6e3371cd        functions/webhookstash:latest   "fwatchdog"              9 minutes ago       Up 9 minutes (healthy)  

In [ ]:
docker $(docker-machine config swnode1) ps

In [53]:
docker $(docker-machine config swnode2) ps

CONTAINER ID        IMAGE                                   COMMAND                CREATED             STATUS                    PORTS               NAMES
aca89e7c16bb        alexellis2/faas-dockerhubstats:latest   "/usr/bin/fwatchdog"   5 minutes ago       Up 5 minutes                                  func_hubstats.1.3q4f3en9pytsqb84xsixkmko0
dccd9d7bb8a9        alexellis2/faas-markdownrender:latest   "fwatchdog"            11 minutes ago      Up 11 minutes             8080/tcp            func_markdown.1.o6mb49suwzpb4dwioqxi0akl9
32b48783aa8d        functions/alpine:health                 "fwatchdog"            11 minutes ago      Up 11 minutes (healthy)                       func_decodebase64.1.hrtugjv0j6wi0r50w9j0bishl
52e917b6ed55        functions/alpine:health                 "fwatchdog"            11 minutes ago      Up 11 minutes (healthy)                       func_echoit.1.or0dk7g02swe37q3kfudap2xb
21e0ae962efe        functions/alpine:health                 "fwatchdog"        

In [47]:
docker node inspect --pretty swnode1


ID:			21m2hzus2bqe7y81hkstiyeo4
Labels:
Hostname:		swnode1
Joined at:		2017-10-25 07:30:51.61060573 +0000 utc
Status:
 State:			Ready
 Availability:		Drain
 Address:		192.168.99.101
Platform:
 Operating System:	linux
 Architecture:		x86_64
Resources:
 CPUs:			1
 Memory:		995.8 MiB
Plugins:
  Log:		awslogs, fluentd, gcplogs, gelf, journald, json-file, logentries, splunk, syslog
  Network:		bridge, host, macvlan, null, overlay
  Volume:		local
Engine Version:		17.10.0-ce
Engine Labels:
 - provider = virtualbox

In [48]:
docker node update --availability active swnode1

swnode1

In [49]:
docker node inspect --pretty swnode1


ID:			21m2hzus2bqe7y81hkstiyeo4
Labels:
Hostname:		swnode1
Joined at:		2017-10-25 07:30:51.61060573 +0000 utc
Status:
 State:			Ready
 Availability:		Active
 Address:		192.168.99.101
Platform:
 Operating System:	linux
 Architecture:		x86_64
Resources:
 CPUs:			1
 Memory:		995.8 MiB
Plugins:
  Log:		awslogs, fluentd, gcplogs, gelf, journald, json-file, logentries, splunk, syslog
  Network:		bridge, host, macvlan, null, overlay
  Volume:		local
Engine Version:		17.10.0-ce
Engine Labels:
 - provider = virtualbox

In [50]:
docker $(docker-machine config swnode1) ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES

In [66]:
NB_time_taken

Started at Wed May 17 19:05:14 CEST 2017   [1495040345]
Ended   at Wed May 17 19:05:14 CEST 2017   [1495040714]
Took 369 secs [0h06m09]
